In [1]:
import os
import glob
import time
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
from externel import seaborn as sns

def bin_CT(img, n_bins=1024):
#     data_vector = np.ravel(img)
    data_max = np.amax(data_vector)
    data_min = np.amin(data_vector)
    # print(data_max, data_min)
    data_squeezed = (data_vector-data_min)/(data_max-data_min)
    data_extended = data_squeezed * n_bins
    data_discrete = data_extended // 1
#     print(data_discrete.shape)
    return np.asarray(list(data_discrete), dtype=np.int64)

train_dict = {}
train_dict["time_stamp"] = time.strftime("%Y-%m-%d_%H:%M:%S", time.localtime())
train_dict["project_name"] = "pixel_correlation"
train_dict["save_folder"] = "./project_dir/"+train_dict["project_name"]+"/"

train_dict["folder_X"] = "./data_dir/norm_MR/discrete/"
train_dict["folder_Y"] = "./data_dir/norm_CT/discrete/"

X_list = sorted(glob.glob(train_dict["folder_X"]+"*.nii.gz"))
Y_list = sorted(glob.glob(train_dict["folder_Y"]+"*.nii.gz"))
print(Y_list[:5])

['./data_dir/norm_CT/discrete/NORM_001.nii.gz', './data_dir/norm_CT/discrete/NORM_002.nii.gz', './data_dir/norm_CT/discrete/NORM_003.nii.gz', './data_dir/norm_CT/discrete/NORM_004.nii.gz', './data_dir/norm_CT/discrete/NORM_005.nii.gz']


In [3]:
n_bin = 128
loc_x, loc_y, loc_z = (194, 126, 80)
dx, dy, dz = (1, 1, 1)
n_X = 20
n_Y = 5
dist_X = np.zeros((n_bin, (dx*2+1)*(dy*2+1)*(dz*2+1)))
dist_Y = np.zeros((n_bin, (dx*2+1)*(dy*2+1)*(dz*2+1)))
src_file = nib.load("./data_dir/norm_MR/NORM_097.nii.gz")
src_data = src_file.get_fdata()
dst_data = np.zeros((256, 256))

In [6]:
from scipy.special import kl_div
from sklearn.metrics import mean_squared_error as mse

def kl_div_scalar(X, Y):
    return np.sum(kl_div(X, Y))

In [8]:
dist_X = np.zeros((n_bin, (dx*2+1)*(dy*2+1)*(dz*2+1)))
dist_Y = np.zeros((n_bin, (dx*2+1)*(dy*2+1)*(dz*2+1)))

for file_path in Y_list[:n_X]:
    Y_file = nib.load(file_path)
    Y_data = Y_file.get_fdata()
    Y_cube = Y_data[loc_x-dx:loc_x+dx+1, loc_y-dy:loc_y+dy+1, loc_z-dz:loc_z+dz+1]
    Y_flat = np.ravel(Y_cube)
    Y_elem = np.unique(Y_cube)
    for cnt_elem, elem in enumerate(Y_elem):
        loc_elem = np.where(Y_flat == elem)[0]
        for idx_elem in loc_elem:
            dist_Y[int(elem), idx_elem] += 1

            
for file_path in X_list[:n_Y]:
    X_file = nib.load(file_path)
    X_data = X_file.get_fdata()
    X_cube = X_data[loc_x-dx:loc_x+dx+1, loc_y-dy:loc_y+dy+1, loc_z-dz:loc_z+dz+1]
    X_flat = np.ravel(X_cube)
    X_elem = np.unique(X_cube)
    for cnt_elem, elem in enumerate(X_elem):
        loc_elem = np.where(X_flat == elem)[0]
        for idx_elem in loc_elem:
            dist_X[int(elem), idx_elem] += 1

            
X_nonzero = []
for idx in range(n_bin):
    row_sum = np.sum(dist_X[idx, :])
    if row_sum > 0:
        dist_X[idx, :] += 1e-6
        row_sum = np.sum(dist_X[idx, :])
        dist_X[idx, :] /=row_sum
        X_nonzero.append(idx)

        
Y_nonzero = []
for idx in range(n_bin):
    row_sum = np.sum(dist_Y[idx, :])
    if row_sum > 0:
        dist_Y[idx, :] /= row_sum
        Y_nonzero.append(idx)
        
        
dist_YX = np.zeros((len(X_nonzero), len(Y_nonzero)))
for cnt_x, elem_X in enumerate(X_nonzero):
    for cnt_y, elem_Y in enumerate(Y_nonzero):
        dist_YX[cnt_x, cnt_y] = kl_div_scalar(dist_Y[elem_Y, :], dist_X[elem_X, :])

        
for cnt_x, elem_X in enumerate(X_nonzero):
    Y_dist_x = dist_YX[cnt_x, :]
    Y_minKL = np.amin(Y_dist_x)
    Y_minKL_loc = np.where(Y_dist_x == Y_minKL)[0][0]
    print(elem_X, Y_nonzero[Y_minKL_loc])

2 64
3 19
4 32
5 78
6 103
7 59
8 17
9 6
10 65
11 52
12 57
13 58
14 5
15 32
16 19
17 66
18 37
19 59
20 52
21 17
23 64
25 32
26 89
27 37
29 93
32 17
35 90
36 88
38 59


In [29]:
print(X_cube.shape)
grad_3d = np.gradient(X_cube)
grad_magn = grad_3d[0]**2 + grad_3d[1]**2 + grad_3d[2]**2
grad_magn = np.sqrt(grad_magn)
grad_magn += 1e-6
print(grad_magn)

(3, 3, 3)
[[[4.58257669e+00 5.38516581e+00 5.19615342e+00]
  [1.73205181e+00 2.54951076e+00 3.35410297e+00]
  [1.00000100e+00 1.50000100e+00 2.44949074e+00]]

 [[2.69258340e+00 2.73861379e+00 2.44949074e+00]
  [7.07107781e-01 1.22474587e+00 1.50000100e+00]
  [1.00000000e-06 5.00001000e-01 1.11803499e+00]]

 [[2.44949074e+00 1.11803499e+00 1.41421456e+00]
  [1.50000100e+00 1.00000100e+00 1.41421456e+00]
  [1.41421456e+00 1.41421456e+00 2.23606898e+00]]]


In [25]:
grad_magn_x = np.arccos(np.asarray(grad_3d[0]) / grad_magn)
grad_magn_y = np.arccos(np.asarray(grad_3d[1]) / grad_magn)
grad_magn_z = np.arccos(np.asarray(grad_3d[2]) / grad_magn)

In [26]:
grad_magn_x

array([[[2.63185258, 2.76108581, 2.86594917],
        [2.18627563, 2.47262808, 2.67794445],
        [3.14017844, 2.30052339, 1.99133048]],

       [[2.76108535, 2.7210575 , 2.52611237],
        [2.35619308, 2.52611179, 2.30052339],
        [1.57079633, 1.57079633, 1.10714917]],

       [[1.99133048, 1.57079633, 0.78539887],
        [1.57079633, 1.57079633, 0.78539887],
        [0.78539887, 0.78539887, 0.4636485 ]]])

In [27]:
grad_magn_y

array([[[2.02242966e+00, 1.95130263e+00, 1.76445459e+00],
        [2.18627563e+00, 2.19981084e+00, 2.03444379e+00],
        [1.57079633e+00, 2.30052339e+00, 2.52611237e+00]],

       [[1.19029010e+00, 1.19700430e+00, 1.15026217e+00],
        [7.85399578e-01, 1.15026236e+00, 1.23095965e+00],
        [1.57079633e+00, 1.57079633e+00, 1.57079633e+00]],

       [[6.15480286e-01, 4.63649398e-01, 7.85398871e-01],
        [1.15470022e-03, 1.41421297e-03, 7.85398871e-01],
        [7.85398871e-01, 7.85398871e-01, 1.10714894e+00]]])

In [28]:
grad_magn_z

array([[[1.3508084 , 1.57079633, 1.76445459],
        [0.95531703, 1.37340085, 1.57079633],
        [1.57079633, 1.910633  , 1.99133048]],

       [[1.57079633, 1.75440027, 1.99133048],
        [1.57079633, 1.9913303 , 2.30052339],
        [1.57079633, 3.13959266, 2.67794326]],

       [[1.15026217, 1.10714917, 1.57079633],
        [1.57079633, 1.57079633, 1.57079633],
        [1.57079633, 1.57079633, 1.57079633]]])